In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from evaluate_data import read_df, integrate_yaw
from dynamic_analysis import add_pose_to_df
from wall_analysis import get_psd

exp_name = '2020_11_23_wall2'; 
#fname = f'results/{exp_name}_simulated.pkl'
fname = f'results/{exp_name}_real.pkl'

try:
    df_total = pd.read_pickle(fname)
    frequencies = df_total.iloc[0].frequencies
except:
    print('could not read', fname)
    df_total = pd.DataFrame(columns=['signals_f', 'degree', 'yaw', 'distance', 'source', 'psd', 'spec', 'frequencies'])

    params = dict(
      props = False,
      snr = False,
      motors = False,
      exp_name = exp_name
    )
    
    for degree in [0, 27, 54, 81, 360]:
        for distance in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]:
            for source in ['mono4125', None, 'sweep']:
                try:
                    params['degree'] = degree
                    params['distance'] = distance
                    params['source'] = source
                    params['appendix'] = ""
                    if distance in [10, 30, 50]:
                        params['appendix'] = "_new"
                    df, df_pos = read_df(**params)
                except Exception as e:
                    continue 

                # detect index decrease (happens when two csv files are concatenated)
                sign = np.sign(df['index'].values[1:] - df['index'].values[:-1])
                if np.any(sign < 0):
                    index = np.where(sign<0)[0][-1]
                    print('Warning: found multiple start indices, start at', index)
                    df = df.iloc[index:]
                    index_start = df.iloc[0]['index']
                    df_pos = df_pos.loc[df_pos.index >= index_start]

                signals_f = np.array([*df.signals_f.values]) # n_times x n_mics x n_freqs
                frequencies_matrix = np.array([*df.loc[:,'frequencies']])
                frequencies = frequencies_matrix[0, :]
                assert not np.any(np.any(frequencies_matrix - frequencies[None, :], axis=0))

                if degree == 360:
                    add_pose_to_df(df, df_pos, max_allowed_lag_ms=50)
                    yaw = integrate_yaw(df.timestamp.values, df.yaw_rate_deg.values)
                else:
                    yaw = np.full(len(df), -degree)

                spec = np.sum(np.abs(signals_f), axis=1)

                psd = get_psd(signals_f, frequencies, fname='real')

                df_total.loc[len(df_total), :] = dict(
                    degree=degree,
                    yaw=yaw,
                    distance=distance,
                    source=str(source),
                    signals_f=signals_f,
                    frequencies=frequencies,
                    spec=spec,
                    psd=psd
                )

    fname = f'results/{exp_name}_real.pkl'
    pd.to_pickle(df_total, fname)
    print('saved as', fname)

In [ ]:
#for deg in [0, 27, 54, 81]:
    #df_total.loc[df_total.degree==deg, 'source'] = 'sweep'

# Fixed angle analysis

In [ ]:
try:
    row = df_total[(df_total.distance==90) & (df_total.degree==81) & (df_total.source=='sweep')].iloc[0]
    spec = np.sum(np.abs(row.signals_f), axis=1)

    plt.figure()
    times = np.arange(spec.shape[0])
    plt.pcolormesh(times, frequencies, np.log10(spec.T))

    psd = get_psd(row.signals_f, ax=plt.gca())

    plt.figure()
    for i_mic in range(psd.shape[0]):
        plt.semilogy(frequencies, np.abs(psd[i_mic, :]), label=f"mic{i_mic}")
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD')
    plt.title(f'degree {degree}, distance {distance}')
except ValueError:
    raise
except:
    print('did not find 90, 81, sweep in')
    print(df_total.distance.unique())
    print(df_total.degree.unique())
    print(df_total.source.unique())

In [ ]:
degrees = [d for d in df_total.degree.unique() if d != 360][:2]
distances = df_total.distance.unique()[:3]
source = 'sweep'

fig, axs = plt.subplots(len(distances), len(degrees), sharex=True, sharey=True)
fig.set_size_inches(10, 10*axs.shape[0]/axs.shape[1])

fig_psd, axs_psd = plt.subplots(len(distances), len(degrees), sharex=True, sharey=True)
fig_psd.set_size_inches(10, 10*axs.shape[0]/axs.shape[1])
for i, distance in enumerate(distances):
    for j, degree in enumerate(degrees):
        df_this = df_total.loc[(df_total.distance == distance)
                               & (df_total.degree == degree)
                               & (df_total.source == source)]
        row = df_this.iloc[0]
        spec = row.spec
        psd = row.psd
        if psd is None:
            psd = get_psd(row.signals_f)
        
        axs[i, j].pcolormesh(range(spec.shape[0]), frequencies, np.log10(spec.T))
        for i_mic in range(psd.shape[0]):
            axs_psd[i, j].semilogy(frequencies, np.abs(psd[i_mic, :]), label=f"mic{i_mic}")
        axs[0, j].set_title(f'{degree} deg')
        axs_psd[0, j].set_title(f'{degree} deg')
    axs[i, 0].set_ylabel(f'{distance} cm')
    axs_psd[i, 0].set_ylabel(f'{distance} cm')
    
[axs[-1, j].set_xlabel(f'time idx') for j in range(len(degrees))]
[axs_psd[-1, j].set_xlabel(f'frequency [Hz]') for j in range(len(degrees))]

In [ ]:
from constants import SPEED_OF_SOUND

distance_ref = 90
distances = df_total.distance.unique()[:3]
source = 'sweep'
n_mics = df_ref.iloc[0].signals_f.shape[1]
print('n_mics:', n_mics)
    
for degree in [0, 27, 54, 81][:2]:
    
    df_ref = df_total.loc[(df_total.distance == distance_ref)
                           & (df_total.degree == degree)
                           & (df_total.source == source)]

    fig, axs = plt.subplots(1, n_mics, sharex=True, sharey=True)
    fig.set_size_inches(15, 5)
    
    for mic in range(n_mics):
        psd_ref = df_ref.iloc[0].psd[mic] 
        for i, distance in enumerate(distances):
            df_this = df_total.loc[(df_total.distance == distance)
                                   & (df_total.degree == degree)
                                   & (df_total.source == source)]
            axs[mic].semilogy(frequencies, df_this.iloc[0].psd[mic], label=distance, color=f"C{i}")
            #axs[mic].semilogy(frequencies, df_this.iloc[0].psd[mic] / psd_ref, label=distance, color=f"C{i}")
            #axs[mic].plot(frequencies, df_this.iloc[0].psd[mic] - psd_ref, label=distance, color=f"C{i}")

        #axs[mic].set_xlim(min(frequencies), max(frequencies))
        axs[mic].set_xlim(2000,  max(frequencies))
        axs[mic].set_title(f"mic{mic}")
    
    fig.suptitle(degree)
    axs[mic].legend()

In [ ]:
#chosen_frequencies = frequencies[[10, 20, 30]]
chosen_frequencies = frequencies[[30]]
print(chosen_frequencies)

distance_ref = 90
mic = 0

for degree in [0, 27, 54, 81][:2]:
    distances = df_total.distance.unique()
    source = 'sweep'

    fig, ax = plt.subplots()
    fig.set_size_inches(15, 5)
    for i, distance in enumerate(distances):
        df_this = df_total.loc[(df_total.distance == distance)
                               & (df_total.degree == degree)
                               & (df_total.source == source)]
        row = df_this.iloc[0]
        for f, freq in enumerate(chosen_frequencies):
            chosen_idx = np.where(frequencies == freq)[0][0]
            ax.scatter(distance, row.psd[mic, chosen_idx], color=f"C{f}")
    ax.set_title(degree)
    #ax.set_yscale('log')
    ax.legend(chosen_frequencies)
    ax.set_xlabel('distance [cm]')

# Moving analysis

In [ ]:
#freq = 4125
#source = 'None'
source = 'mono4125'
degree = 360

distances = df_total.distance.unique()[:3]
chosen_idx = np.where(frequencies == freq)[0][0]

averages = []
for i, distance in enumerate(distances):
    fig, axs = plt.subplots(2)
    fig.set_size_inches(15, 5)
    
    ax = axs[0]
    df_this = df_total.loc[(df_total.distance == distance)
                           & (df_total.degree == degree)]
    df_this = df_this.loc[df_this.source == source]
                           
    signals_f = df_this.iloc[0].signals_f
    yaw =  df_this.iloc[0].yaw
    spec = df_this.iloc[0].spec

    for j in range(signals_f.shape[1]):
        ax.semilogy(range(signals_f.shape[0]), np.abs(signals_f[:, j, chosen_idx]), color=f"C{j}", label=f"mic{j}")

    yaw[np.isnan(yaw)] = 0
    axs[1].plot(range(signals_f.shape[0]), yaw)
    for deg in -np.arange(1, 5)*90:
        index = np.nanargmin(np.abs(yaw-deg))
        axs[1].axvline(x=index, color='C1')
        ax.axvline(x=index, color='C1')
        
    min_avg = 200
    max_avg = 300
    averages.append(np.sum(spec[min_avg:max_avg, chosen_idx], axis=0))
        
    ax.set_title(distance)
    ax.set_ylim(0.05, 10)
    ax.legend()
    [ax.grid() for ax in axs]

In [ ]:
plt.figure()
plt.scatter(distances, averages)
plt.xlabel('distance [cm]')
plt.title(f'average PSD at {freq} Hz')